# Descriptors
* non-data descriptors: only implements __get__
* data descriptors: implements __set__ and/or __delete__

In [10]:
from datetime import datetime

class TimeUTC:
    def __get__(self, instance, owner_class):
        return datetime.utcnow().isoformat()

class Logger:
    current_time = TimeUTC()

print(Logger.__dict__)
l = Logger()
print(l.current_time)
print(Logger.current_time)

{'__module__': '__main__', 'current_time': <__main__.TimeUTC object at 0x0000016CC56A0508>, '__dict__': <attribute '__dict__' of 'Logger' objects>, '__weakref__': <attribute '__weakref__' of 'Logger' objects>, '__doc__': None}
2021-05-14T01:08:10.168455
2021-05-14T01:08:10.168455


# Descriptor Protocol: getters and setters
* when multiple instances of a Class that uses a Descriptor, 

In [12]:
from datetime import datetime

class TimeUTC:
    def __get__(self,instance,owner_class):
        print(f'__get__ called, self={self}, instance={instance}, owner_class={owner_class}')
        return datetime.utcnow().isoformat()

class Logger1:
    current_time = TimeUTC()

class Logger2:
    current_time = TimeUTC()

# when Descriptor is called from class, the instance is None
print(Logger1.current_time)
print(Logger2.current_time)
print('\n')
l1 = Logger1()
l2 = Logger2()
v1 = Logger1()
# when Descriptor is called from Instance, the instance is the object instance
print(l1.current_time)
print(v1.current_time)
print(l2.current_time)


__get__ called, self=<__main__.TimeUTC object at 0x000001E87A460508>, instance=None, owner_class=<class '__main__.Logger1'>
2021-05-14T14:08:33.879060
__get__ called, self=<__main__.TimeUTC object at 0x000001E87A460488>, instance=None, owner_class=<class '__main__.Logger2'>
2021-05-14T14:08:33.880071


__get__ called, self=<__main__.TimeUTC object at 0x000001E87A460508>, instance=<__main__.Logger1 object at 0x000001E87A460608>, owner_class=<class '__main__.Logger1'>
2021-05-14T14:08:33.880071
__get__ called, self=<__main__.TimeUTC object at 0x000001E87A460508>, instance=<__main__.Logger1 object at 0x000001E87A4C8908>, owner_class=<class '__main__.Logger1'>
2021-05-14T14:08:33.880071
__get__ called, self=<__main__.TimeUTC object at 0x000001E87A460488>, instance=<__main__.Logger2 object at 0x000001E87A4606C8>, owner_class=<class '__main__.Logger2'>
2021-05-14T14:08:33.880071


In [15]:
from datetime import datetime

class TimeUTC:
    def __get__(self,instance,owner_class):
        if instance is None:
            return  self
        else:    
            return datetime.utcnow().isoformat()

class Logger1:
    current_time = TimeUTC()

class Logger2:
    current_time = TimeUTC()

print(Logger1.current_time)
l1 = Logger1()
print('\n')
print(l1.current_time)



2021-05-14T14:14:08.684401


In [25]:
class IntegerValue:
    def __set__(self,instance,value):
        print(f'__set__ called, instance={instance}, value={value}')
        self._value = value

    def __get__(self,instance,owner_class):
        if instance is None:
            print('__get__ called from class')
            return self
        else:
            return self._value

class Point2D:
    x = IntegerValue()
    y = IntegerValue()

print(Point2D.x)
p1 = Point2D()
p1.x = 1.1
p1.y = 2.2

p2 = Point2D()
p2.x = 10
print(p2.x)
print(p1.x)

__get__ called from class
__set__ called, instance=<__main__.Point2D object at 0x000001E87AF84508>, value=1.1
__set__ called, instance=<__main__.Point2D object at 0x000001E87AF84508>, value=2.2
__set__ called, instance=<__main__.Point2D object at 0x000001E87AF84FC8>, value=10
10
10


# Using as Instance Properties
* watch out for memory leaks!

In [27]:
class IntegerValue:
    def __set__(self,instance,value):
        instance.stored_value = int(value)

    def __get__(self,instance,owner_class):
        if instance is None:
            return self
        return getattr(instance,'stored_value',None)

class Point1D:
    x = IntegerValue()

class Point2D:
    x = IntegerValue()
    y = IntegerValue()

p1 = Point1D()
p2 = Point1D()
p3 = Point2D()

p1.x = 10.1
p2.x = 20.2
print(p1.x,p2.x)

p3.x = 10
p3.y = 20
print(p3.x,p3.y)

10 20
20 20


In [30]:
class IntegerValue:
    def __init__(self,name):
        self.stored_name = '_' + name

    def __set__(self,instance,value):
        setattr(instance,self.stored_name,value)

    def __get__(self,instance,owner_class):
        if instance is None:
            return self
        return getattr(instance,self.storage_name,None)


class Point2D:
    x = IntegerValue('x')
    y = IntegerValue('y')

p1 = Point2D()
p1.x = 10.1
p1.y = 20.2
p2.x = 23
p2.y = 32
print(p1.__dict__)
print(p2.__dict__)

{'_x': 10.1, '_y': 20.2}
{'stored_value': 23, 'y': 32}


# strong and weak references
* weeak reference are use to prevent memory Leak

In [52]:
import ctypes

# garbage collection
def ref_count(address):
    return ctypes.c_long.from_address(address).value

class Person:
    def __init__(self,name):
        self.name = name
    def __repr__(self):
        return f'Person(name={self.name})'

p1 = Person('john')
p2 = p1

print(p1 is p2, hex(id(p1)), hex(id(p2)))
p1_id = id(p1)
print(p1_id)
print(ref_count(p1_id))
del p1

True 0x1e87ae88648 0x1e87ae88648
2098006099528
2


In [63]:
import weakref

def ref_count(address):
    return ctypes.c_long.from_address(address).value

class Person:
    def __init__(self,name):
        self.name = name
    def __repr__(self):
        return f'Person(name={self.name})'

p1 = Person('john')
p2 = p1
print(ref_count(id(p1)))
weak1 = weakref.ref(p1)
print(weak1 is p1)
print(weak1() is p1)
# never execute weak1() or you will create strong references instead us print
print(f'this is the correct way: {weak1()}')
print(ref_count(id(p1)))
del p2
print(ref_count(id(p1)))
del p1
print(f'garbage colletor works: {weak1}')
result = weak1()
print(result)

2
False
True
this is the correct way: Person(name=john)
2
1
garbage colletor works: <weakref at 0x000001E8795B8B88; dead>
None


* not many types support weak references

In [65]:
import weakref
l = 'python'
try:
    w = weakref.ref(l)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'str' object


In [77]:
import weakref

def ref_count(address):
    return ctypes.c_long.from_address(address).value

class Person:
    def __init__(self,name):
        self.name = name
    def __repr__(self):
        return f'Person(name={self.name})'

p1 = Person('john')
d = weakref.WeakKeyDictionary()
print(ref_count(id(p1)))
n = {p1:'john'}
print(ref_count(id(p1)))
del n
print(ref_count(id(p1)))
d[p1]='john'
print(ref_count(id(p1)))
d2 = weakref.WeakKeyDictionary()
d2[p1]='john'
print(f'this is the number of strong references: {ref_count(id(p1))}')
print(f'this is the number of weak references: {weakref.getweakrefcount(p1)}')
print(list(d.keyrefs()))
del p1
print(list(d.keyrefs()))

1
2
1
1
this is the number of strong references: 1
this is the number of weak references: 2
[<weakref at 0x000001E87C61BEA8; to 'Person' at 0x000001E87C775888>]
[]


# Instance Properties

In [83]:
import weakref

class IntegerValue:
    def __init__(self):
        self.values =weakref.WeakKeyDictionary()
    def __set__(self,instance,value):
        self.values[instance] = int(value)
    def __get__(self,instance,owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

class Point:
    x = IntegerValue()

p = Point()
print(hex(id(p)))
p.x = 100.1
print(p.x)
print(Point.x.values.keyrefs())
del p
print(Point.x.values.keyrefs())

0x1e87c596c08
100
[<weakref at 0x000001E879880A98; to 'Point' at 0x000001E87C596C08>]
[]


# Descriptors with not hashable classes that has __slots__

In [99]:
import weakref

class ValidString:
    def __init__(self,min_length=0,max_length=255):
        self.data = {}
        self._min_length = min_length
        self._max_length = max_length
    def __set__(self,instance,value):
        if not isinstance(value,str):
            raise ValueError('Value must be a string')
        if len(value) < self._min_length:
            raise ValueError(f'Value should be at least {self._min_length} characters')
        if len(value) > self._max_length:
            raise ValueError(f'Value cannot exceed {self._min_length} characters')
        self.data[id(instance)] = (weakref.ref(instance, self._finalize_instance),value)
    def __get__(self,instance,owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.data.get(id(instance))
            return value_tuple[1]
    def _finalize_instance(self,weak_ref):
        reverse_lookup = [key for key,value in self.data.items() if value[0] is weak_ref]
        if reverse_lookup:
            key = reverse_lookup[0]
            del self.data[key]

class Person:
    __slots__ = '__weakref__',
    first_name = ValidString(1,100)     
    last_name = ValidString(1,100)     
    def __eq__(self,other):
        return(
                isinstance(other,Person) and
                self.first_name == other.first_name and
                self.last_name == other.last_name
            )

class BankAccount:
    __slots__ = '__weakref__',
    account_number = ValidString(5,255)
    def __eq__(self,other):
        return isinstance(other, BankAccount) and self.account_number == other.account_number

p1 = Person()
try:
    p1.first_name = ''
except ValueError as ex:
    print(ex)

p2 = Person()
p1.first_name, p1.last_name = 'john', 'cardona'
p2.first_name, p2.last_name = 'Andrea', 'Melo'

b1, b2 = BankAccount(), BankAccount() 
b1.account_number,b2.account_number = 'savings', 'checking'
print(p1.first_name)
print(p1.last_name)
print(p2.first_name)
print(p2.last_name)

print(b1.account_number)
print(b2.account_number)
print(Person.first_name.data, '\n')
print(Person.last_name.data, '\n')
print(BankAccount.account_number.data, '\n')

del p1
del p2
del b1
del b2

print(Person.first_name.data)


Value should be at least 1 characters
john
cardona
Andrea
Melo
savings
checking
{2098007106472: (<weakref at 0x000001E87A597688; to 'Person' at 0x000001E87AF7E3A8>, 'john'), 2098007105608: (<weakref at 0x000001E87A5978B8; to 'Person' at 0x000001E87AF7E048>, 'Andrea')} 

{2098007106472: (<weakref at 0x000001E87A597368; to 'Person' at 0x000001E87AF7E3A8>, 'cardona'), 2098007105608: (<weakref at 0x000001E87C6D3548; to 'Person' at 0x000001E87AF7E048>, 'Melo')} 

{2097982995096: (<weakref at 0x000001E87C3B00E8; to 'BankAccount' at 0x000001E87987FA98>, 'savings'), 2097982993752: (<weakref at 0x000001E87C3B08B8; to 'BankAccount' at 0x000001E87987F558>, 'checking')} 

{}


# The __set_name__ method

In [7]:
class ValidString:
    def __init__(self, min_length=None):
        self.min_length = min_length
    
    def __set_name__(self,owner_class,property_name):
        self.property_name = property_name
    
    def __set__(self,instance,value):
        if not isinstance(value,str):
            raise ValueError(f'{self.property_name} must be a String.')
        if self.min_length is not None and len(value) < self.min_length:
            raise ValueError(
                                f'{self.property_name} must be at least {self.min_length} characters.'
                            )
        instance.__dict__[self.property_name] = value
    
    def __get__(self,instance,owner_class):
        if instance is None:
            return self
        print('__get__ called!')
        return instance.__dict__.get(self.property_name,None)

class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)

p = Person()
print(p.__dict__)
try:
    p.first_name = 'john'
    p.last_name = 'c'
except ValueError as  ex:
    print(ex)

p.first_name = 'john'
p.last_name = 'Cardona'
print(p.__dict__)
print(p.first_name)



{}
last_name must be at least 2 characters.
{'first_name': 'john', 'last_name': 'Cardona'}
__get__ called!
john


# Property value lookup resolution
* Non-Data descriptors: first try to lookup at __dict__ if nothing is found it uses __get__
* Data descriptors: always use __get__

In [14]:
# non-data Descriptor
class Non_data_Descriptor:
    def __get__(self,instance,owner_class):
        print('__get__ called!')
class Main:
    descriptor = Non_data_Descriptor()

d = Main()
print(d.descriptor)
print(d.__dict__, '\n')
d.__dict__['descriptor'] = 'hello!'
print(d.descriptor)


__get__ called!
None
{} 

hello!


In [23]:
# data Descriptor
class Data_Descriptor:
    def __set__(self,instance,value):
        print('__set__ called!')
    def __get__(self,instance,owner_class):
        print('__get__ called!')

class Main:
    x = Data_Descriptor()
p = Main()
p.x = 100
p.x
p.__dict__
p.__dict__['x'] = 'hello!'
print(p.__dict__)
print(p.x)



__set__ called!
__get__ called!
{'x': 'hello!'}
__get__ called!
None


# properties and descriptors
* property objects are data descriptors

In [27]:
# using decorators
from numbers import Integral

class Person:
    @property
    def age(self):
        return getattr(self,'_age',None)
    @age.setter
    def age(self, value):
        if not isinstance(value, Integral):
            raise ValueError('age: must be an integer.')
        if value < 0:
            raise ValueError('age: must be an non-negative integer.')
        self._age = value

p = Person()

try:
    p.age = -10
except ValueError as ex:
    print(ex)

p.age = 10
print(p.__dict__)

age: must be an non-negative integer.
{'_age': 10}


In [37]:
from numbers import Integral

class Person:

    def get_age(self):
        return getattr(self,'_age',None)

    def set_age(self, value):
        if not isinstance(value, Integral):
            raise ValueError('age: must be an integer.')
        if value < 0:
            raise ValueError('age: must be an non-negative integer.')
        self._age = value
    age = property(fget=get_age,fset=set_age)

prop = Person.age
print(hasattr(prop, '__get__'))
print(hasattr(prop, '__set__'))
print(hasattr(prop, '__delete__'))
print('\n')
p = Person()
p.age = 10
print(p.__dict__)
p.__dict__['age'] = 100
print(p.__dict__)
print(p.age)

True
True
True


{'_age': 10}
{'_age': 10, 'age': 100}
10


# Replicate property decorator

In [40]:
class MakeProperty:
    def __init__(self,fget=None,fset=None):
        self.fget = fget
        self.fset = fset

    def __set_name__(self,owner_class,prop_name):
        self.prop_name = prop_name
    
    def __get__(self, instance, owner_class):
        print('__get__ called!')
        if instance is None:
            return self
        if self.fget is None:
            raise AttributeError(f'{self.prop_name} is not readable.')
        return self.fget(instance)

    def __set__(self, instance, value):
        print('__set__ called!')
        if self.fset is None:
            raise AttributeError(f'{self.prop_name} is not writable.')
        self.fset(instance, value)

class Person:
    def get_name(self):
        print('get_name called!')
        return getattr(self, '_name', None)
    
    def set_name(self, value):
        print('set_name called!')
        self._name = value
    
    name = MakeProperty(fget=get_name, fset=set_name)

p = Person()
p.name = 'john'
p.name


__set__ called!
set_name called!
__get__ called!
get_name called!


'john'

In [45]:
class MakeProperty:
    def __init__(self,fget=None,fset=None):
        self.fget = fget
        self.fset = fset

    def __set_name__(self,owner_class,prop_name):
        self.prop_name = prop_name
    
    def __get__(self, instance, owner_class):
        print('__get__ called!')
        if instance is None:
            return self
        if self.fget is None:
            raise AttributeError(f'{self.prop_name} is not readable.')
        return self.fget(instance)

    def __set__(self, instance, value):
        print('__set__ called!')
        if self.fset is None:
            raise AttributeError(f'{self.prop_name} is not writable.')
        self.fset(instance, value)
    
    def setter(self, fset):
        self.fset = fset
        return self

class Person:
    @MakeProperty
    def age(self):
        return getattr(self, '_age', None)
    @age.setter
    def age(self,value):
        self._age = value

p = Person()
p.age = 10
print(p.age)
print(p.__dict__)

__set__ called!
__get__ called!
10
{'_age': 10}


# Application 1
* create a Data descriptor to make a type validation 

In [48]:
import numbers
class ValidType:
    def __init__(self,type_):
        self._type = type_
    def __set_name__(self,owner_class,prop_name):
        self.prop_name = prop_name
    def __set__(self,instance,value):
        if not isinstance(value, self._type):
            raise ValueError(f'{self.prop_name} must be of type {self._type.__name__}.')
        instance.__dict__[self.prop_name] = value
    def __get__(self,instance,owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name,None)

class Person:
    age = ValidType(int)
    height = ValidType(numbers.Real)
    tags = ValidType(list)
    favorite_food = ValidType(tuple)

p = Person()

try:
    p.age = 10.1
except ValueError as ex:
    print(ex)



age must be of type int.
